In [1]:
import pandas as pd

In [2]:
aqi_df = []
aqi_files_start_year = 2017

for i in range(5) :
    # Load the data into a dataframe
    df = pd.read_csv('raw_data/annual_aqi_by_county_' + str(i + aqi_files_start_year) + '.csv')
    #print(df.info())

    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df[df['state'] == 'California'].drop(columns=['state'])
    
    aqi_df.append(df)

In [3]:
# combine all aqi years dataframes into one
combined_aqi_df = pd.concat(aqi_df, ignore_index=True)
#print(combined_aqi_df)

In [40]:
# combine all aqi years dataframes into one
cleaned_aqi_df = pd.concat(aqi_df, ignore_index=True)
#print(cleaned_aqi_data)
cleaned_aqi_df.to_csv('processed_data/cleaned_aqi.csv')

In [31]:
# clean asthma emergency department visits data

asthma_df = []
asthma_files_start_year = 2017

for i in range(5) :
    # load the data into a dataframe
    df = pd.read_excel('raw_data/Asthma_Emergency_' + str(i + asthma_files_start_year) + '.xlsx')
    #print(df.info())

    # clean up the dataframe
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df.rename(columns={'counties': 'county'}, inplace=True)
    df = df[df['county'] != 'California'] # remove rows with 'California' in county name
    df = df.drop(columns=['lower_95%_limit', 'upper_95%_limit'])  # optional, but cleaner
 
    # add year column and reorder columns to have year after county (like in aqi_df)
    df['year'] = i + asthma_files_start_year
    columns_order = ['county', 'year'] + [col for col in df.columns if col not in ['county', 'year']]
    df = df[columns_order]

    asthma_df.append(df)


In [38]:
# combine all asthma years dataframes into one
cleaned_asthma_df = pd.concat(asthma_df, ignore_index=True)

#print(cleaned_asthma_df)
cleaned_asthma_df.to_csv('processed_data/cleaned_asthma.csv')


In [25]:
# find counties missing

# count number of years county appears in aqi data
year_counts = combined_aqi_df.groupby('county')['year'].nunique()
#print(year_counts.sort_values())

#figure out which counties missing in aqi data
all_counties = combined_asthma_df['county'].unique()
aqi_counties = combined_aqi_df['county'].unique()
#print(len(all_counties), len(aqi_counties))
print(all_counties)

['Alameda' 'Alpine' 'Amador' 'Butte' 'Calaveras' 'Colusa' 'Contra Costa'
 'Del Norte' 'El Dorado' 'Fresno' 'Glenn' 'Humboldt' 'Imperial' 'Inyo'
 'Kern' 'Kings' 'Lake' 'Lassen' 'Los Angeles' 'Madera' 'Marin' 'Mariposa'
 'Mendocino' 'Merced' 'Modoc' 'Mono' 'Monterey' 'Napa' 'Nevada' 'Orange'
 'Placer' 'Plumas' 'Riverside' 'Sacramento' 'San Benito' 'San Bernardino'
 'San Diego' 'San Francisco' 'San Joaquin' 'San Luis Obispo' 'San Mateo'
 'Santa Barbara' 'Santa Clara' 'Santa Cruz' 'Shasta' 'Sierra' 'Siskiyou'
 'Solano' 'Sonoma' 'Stanislaus' 'Sutter' 'Tehama' 'Trinity' 'Tulare'
 'Tuolumne' 'Ventura' 'Yolo' 'Yuba']


In [30]:
missing_counties_aqi = set(all_counties) - set(aqi_counties)
print("Counties missing in AQI data:", missing_counties_aqi)

Counties missing in AQI data: {'Modoc', 'Lassen', 'Alpine', 'Yuba', 'Sierra'}


In [ ]:
# find rows with null values in ashtma data
missing_counties_asthma = combined_asthma_df[combined_asthma_df.isnull().any(axis=1)]

# print counties with null values
print(missing_counties_asthma['county'].unique())

# print the entire rows with null values
print(missing_counties_asthma)

In [ ]:
import itertools

years = sorted(combined_aqi_df['year'].unique())
skeleton = pd.DataFrame(itertools.product(all_counties, years),
                        columns=['county','year'])

merged = skeleton.merge(combined_aqi_df, on=['county','year'], how='left')

# Rows where median_aqi is missing:
gaps = merged[merged['median_aqi'].isna()]

# Count gaps per county
gap_counts = gaps.groupby('county').size().sort_values(ascending=False)
print(gap_counts)

In [29]:
all_counties = ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa', 'Contra Costa',
                'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt', 'Imperial', 'Inyo',
                'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin', 'Mariposa',
                'Mendocino', 'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
                'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito', 'San Bernardino',
                'San Diego', 'San Francisco', 'San Joaquin', 'San Luis Obispo', 'San Mateo',
                'Santa Barbara', 'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou',
                'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare',
                'Tuolumne', 'Ventura', 'Yolo', 'Yuba']

In [41]:
# merge cleaned data sets
merged_data = pd.merge(cleaned_aqi_df, cleaned_asthma_df, 
                      on=['county', 'year'], 
                      how='inner')

In [52]:
print(f"Final dataset: {len(merged_data)} rows, {len(merged_data.columns)} columns")
print(f"Counties covered: {merged_data['county'].nunique()}")
merged_data_timeframe = str(merged_data['year'].min()) + "-" + str(merged_data['year'].max())
print(f"Years covered: ", merged_data_timeframe)

Final dataset: 265 rows, 19 columns
Counties covered: 53
Years covered:  2017-2021


In [54]:
merged_data.to_csv('processed_data/merged_data_' + merged_data_timeframe + '.csv')